In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.ml.feature import StringIndexer
import sys

In [11]:
import pandas as pd

In [2]:
ip_address = '54.214.104.75'
sqlContext = SQLContext(sc)

In [3]:
df_review = sqlContext.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri","mongodb://"+ip_address+"/yelp.review").load()

df_tip = sqlContext.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri","mongodb://"+ip_address+"/yelp.tip").load()


In [4]:
rstdata = sc.textFile('/home/hadoop/Collaborative-Filtering/data/restaurant_ids_final.txt').map(lambda x: (x,1))



In [23]:
ids = pd.read_csv('/home/hadoop/Collaborative-Filtering/data/restaurant_ids_final.txt').values.tolist()

In [30]:
rstdata = sc.parallelize(ids).map(lambda x: (x[0],1))

In [31]:
rstdata.take(4)

[('FweiR9ytUFhQ8hHGY0ySGw', 1),
 ('-VBi01c4m2rWyzUj0rlvKQ', 1),
 ('Vlp68XpPYH0G9NHkgcglvA', 1),
 ('Ry0Sa_sMYdGehltrv7Cx0Q', 1)]

In [32]:
rstdata = rstdata.toDF(['business_id','biz_ix']).select('business_id')

+--------------------+
|         business_id|
+--------------------+
|FweiR9ytUFhQ8hHGY...|
|-VBi01c4m2rWyzUj0...|
|Vlp68XpPYH0G9NHkg...|
|Ry0Sa_sMYdGehltrv...|
|hPjVz5ixa54oIpVcR...|
|2G_6PBM-klbh1u2v2...|
|GD9mTnCht2bog2yb0...|
|E0T-xQJXpM6Hsm-Ee...|
|8u6NUtxSPH3CbLKTQ...|
|qJRZ7eaarbermHS3Z...|
|rslX_CGOBr5m6yOor...|
|dQ11s7taakRn8omBU...|
|qYMnY_A5Qh5wpK26T...|
|RruVXr3L6G5RplZ_P...|
|FS84wtnQv2i1cCO8h...|
|Jxnqwp03AcAeqS4ma...|
|McZz8few5LqWF37KZ...|
|2C-Su_yxiOfojKP5x...|
|bKoLvj4R8OUE2tFiq...|
|hc7bFjQ0JtwfZhKDG...|
+--------------------+
only showing top 20 rows



In [34]:
df_review.persist()
df_tip.persist()
rstdata.persist()

DataFrame[business_id: string]

In [35]:
userMap = df_review.select('user_id').union(df_tip.select('user_id')).distinct()
indexer_userid = StringIndexer(inputCol="user_id", outputCol="user_ix")
userMap = indexer_userid.fit(userMap).transform(userMap)

bizMap = df_review.select('business_id').union(df_tip.select('business_id')).distinct()
indexer_biz = StringIndexer(inputCol="business_id", outputCol="biz_ix")
bizMap = indexer_biz.fit(bizMap).transform(bizMap)
bizMap = rstdata.join(bizMap,on='business_id',how='inner')

In [36]:
df_review_als = df_review.select('user_id','business_id','stars')
df_review_als = df_review_als.join(userMap,on='user_id',how='left_outer').join(bizMap,on='business_id',how='inner')
ALS_baseline_df = df_review_als.select('user_ix','biz_ix','stars')
df_tip_df = df_tip.join(userMap,on='user_id',how='left_outer').join(bizMap,on='business_id',how='inner')
## drop and Save table

userMap.write.parquet("s3://dknsyelp/userMap.parquet", mode='overwrite')
bizMap.write.parquet("s3://dknsyelp/bizMap.parquet", mode='overwrite')
ALS_baseline_df.write.parquet("s3://dknsyelp/ALS_baseline.parquet", mode='overwrite')
df_review.write.parquet("s3://dknsyelp/review.parquet", mode='overwrite')
df_tip_df.write.parquet("s3://dknsyelp/tip.parquet", mode='overwrite')